In [2]:
import pandas as pd
import re
import pickle
import datetime
#import textdistance
import json
import numpy as np
import wikipedia as wiki
path_data= 'C:/root/Universita/project_dmdv/data/'




#some useful functions

#funzione per la sostituzione sulla base di una regex, la funzione nativa va in errore se il valore è null
def regex_replace(regex, sub, string):
    if pd.isnull(string):
        return None
    else:
        return re.sub(regex, sub, string)

#matching sulla base di una regex, la f nativa va in errore se il valore è mancante
def regex_match(regex, string, reverse = False):
    if pd.isnull(string):
        return None
    else:
        return re.match(regex, string)
    
#is_winner
def get_winner(string):
    if string:
        return 'Winner'
    else:
        return 'Nominee'

#funzione per la sostituzione delle stringe contententi solo spazi
def replace_na_string(string):
    str_out = string
    if pd.notnull(string):
        if string.replace(' ', '') == '':
            str_out = None
        else:
            str_out = string.upper()
    return str_out

#estrazione sottostringa (se pd.NoneType, lo slicing di python base va in errore)
def substring(string, start, end):
    str_out = None
    if pd.notnull(string):
        str_out = string[start:end]
    return str_out

#conversione in tipo data ed eventuale pulizia in caso di errori (es. 0008 -> 2008, 20012 -> 2012, ecc)
def get_date(date):
    try:
        date_out = pd.to_datetime(date, format = "%Y-%m-%d")
    except:
        date_list = date.split('-')
        if date_list[0][:2] == '10':
            date_list[0] = '19' + date_list[0][-2:]
        elif date_list[0][:2] == '00':
            date_list[0] = '20' + date_list[0][-2:]
        elif len(date_list[0]) > 4:
            date_list[0] = date_list[0][:2] + date_list[0][-2:]
        date_out = pd.to_datetime('-'.join(date_list), format = "%Y-%m-%d")
    return date_out

#f spostare l'articolo all'inizio della stringa, sul sito web in alcuni casi è in fondo
def move_article(program):
    output_string = program
    if pd.notnull(program):
        if (len(re.findall(r',\s*THE\s*(\([0-9]*\))*\s*$', program.upper())) > 0):
            output_string = 'The ' + re.sub(r',\s*THE\s*(\([0-9]*\))*\s*$', '',  program.upper())
        elif (len(re.findall(r',\s*A\s*(\([0-9]*\))*\s*$', program.upper())) > 0):
            output_string = 'A ' + re.sub(r',\s*A\s*(\([0-9]*\))*\s*$', '',  program.upper())
        elif (len(re.findall(r',\s*AN\s*(\([0-9]*\))*\s*$', program.upper())) > 0):
            output_string = 'An ' + re.sub(r',\s*AN\s*(\([0-9]*\))*\s*$', '',  program.upper())
    return output_string

# AWARDS

### MOVIES

##### Oscar

In [2]:
oscar_data = pd.read_csv(path_data + 'awards/the_oscar_award.csv', encoding = 'UTF-8',
                         dtype = {'program' :'str', 'year_film' : 'float'},
                        converters = {'name' : lambda x: x.split(',')})
#oscar_data['year'] = oscar_data['year'].apply(lambda row: float(row))
oscar_data['award_type'] = 'oscar award'

#conversione del valore bool in stringa
oscar_data['is_winner'] = oscar_data['winner'].apply(lambda row: get_winner(row))

#strutturo i dati in maniera uniforme
oscar_data = oscar_data[['category', 'year_film', 'film', 'name', 'is_winner', 'award_type']]
oscar_data.columns = ['award', 'year', 'program', 'person', 'is_winner', 'award_type']
oscar_data.tail()

,award,year,program,person,is_winner,award_type
10390,WRITING (Original Screenplay),2019.0,Parasite,"[Screenplay by Bong Joon Ho, Han Jin Won; Sto...",Winner,oscar award
10391,JEAN HERSHOLT HUMANITARIAN AWARD,2019.0,NaN,[Geena Davis],Winner,oscar award
10392,HONORARY AWARD,2019.0,NaN,[ David Lynch],Winner,oscar award
10393,HONORARY AWARD,2019.0,NaN,[ Wes Studi],Winner,oscar award
10394,HONORARY AWARD,2019.0,NaN,[ Lina Wertmüller],Winner,oscar award


##### Golden Globe

In [3]:
gglobe_data = pd.read_pickle(path_data + 'awards/gglobe_data.dataset')
gglobe_data.head()

,award,year,program,person,is_winner
0,Best Motion Picture - Drama,2020.0,1917,[],Winner
1,Best Motion Picture - Drama,2020.0,"Irishman, The",[],Nominee
2,Best Motion Picture - Drama,2020.0,Joker,[],Nominee
3,Best Motion Picture - Drama,2020.0,Marriage Story,[],Nominee
4,Best Motion Picture - Drama,2020.0,"Two Popes, The",[],Nominee


In [4]:
award_types = pd.read_excel(path_data + 'custom/award_types.xlsx', sheet_name = 'golden_globe')
gglobe_data = gglobe_data.merge(award_types, on = 'award', how = 'left')
award_types.head()

,award,type
0,Best Motion Picture - Drama,M
1,Best Motion Picture - Musical or Comedy,M
2,Best Performance by an Actress in a Motion Pic...,M
3,Best Performance by an Actor in a Motion Pictu...,M
4,Best Performance by an Actress in a Motion Pic...,M


In [5]:
gglobe_data_movies = gglobe_data.copy()[gglobe_data['type'] == 'M'].iloc[:, :-1]
gglobe_data_movies.head()

,award,year,program,person,is_winner
0,Best Motion Picture - Drama,2020.0,1917,[],Winner
1,Best Motion Picture - Drama,2020.0,"Irishman, The",[],Nominee
2,Best Motion Picture - Drama,2020.0,Joker,[],Nominee
3,Best Motion Picture - Drama,2020.0,Marriage Story,[],Nominee
4,Best Motion Picture - Drama,2020.0,"Two Popes, The",[],Nominee


In [6]:
#anno portato sull'anno di uscita
gglobe_data_movies['year'] = gglobe_data_movies['year'].apply(lambda row: float(row)-1)
gglobe_data_movies['award_type'] = 'golden globe'
gglobe_data_movies.head()

,award,year,program,person,is_winner,award_type
0,Best Motion Picture - Drama,2019.0,1917,[],Winner,golden globe
1,Best Motion Picture - Drama,2019.0,"Irishman, The",[],Nominee,golden globe
2,Best Motion Picture - Drama,2019.0,Joker,[],Nominee,golden globe
3,Best Motion Picture - Drama,2019.0,Marriage Story,[],Nominee,golden globe
4,Best Motion Picture - Drama,2019.0,"Two Popes, The",[],Nominee,golden globe


##### All Award

In [7]:
data_award = pd.concat([oscar_data, gglobe_data_movies]).reset_index().drop(labels = ['index'], axis = 1).reset_index()

#trim
data_award['program'] = data_award['program'].apply(lambda row: str(row).strip() if pd.notnull(row) else None)

#pulizia articolo
data_award['program'] = data_award['program'].apply(lambda row: move_article(row))

In [8]:
#upper del nome del film e set None per le stringhe vuote
data_award['title_upper'] = data_award['program'].apply(lambda row: replace_na_string(row.upper()) if pd.notnull(row) else None)

In [9]:
data_award.head()

,index,award,year,program,person,is_winner,award_type,title_upper
0,0,ACTOR,1927.0,The Noose,[Richard Barthelmess],Nominee,oscar award,THE NOOSE
1,1,ACTOR,1927.0,The Last Command,[Emil Jannings],Winner,oscar award,THE LAST COMMAND
2,2,ACTRESS,1927.0,A Ship Comes In,[Louise Dresser],Nominee,oscar award,A SHIP COMES IN
3,3,ACTRESS,1927.0,7th Heaven,[Janet Gaynor],Winner,oscar award,7TH HEAVEN
4,4,ACTRESS,1927.0,Sadie Thompson,[Gloria Swanson],Nominee,oscar award,SADIE THOMPSON


In [10]:
#split per i premi con nome mancante
data_award_naName = data_award.copy()[pd.isnull(data_award['title_upper'])]
data_award = data_award[pd.notnull(data_award['title_upper'])]

print('Totali: ' + str(data_award_naName.shape[0] + data_award.shape[0]))
print('Missing: ' + str(data_award_naName.shape[0]))
print('Non missing: ' + str(data_award.shape[0]))

Totali: 14331
Missing: 305
Non missing: 14026


In [12]:
last_id_num = data_award.shape[0] + 1

#genero l'id
data_award['award_id'] = ['aw-' + str(i) for i in range(1, last_id_num)]
data_award = data_award.drop(['index'], axis = 1)

In [13]:
data_award.to_pickle(path_data + 'awards/movies_awards.data')

### TV SERIES

##### Golden Globe

In [14]:
gglobe_data = pd.read_pickle(path_data + 'awards/gglobe_data.dataset')
gglobe_data.head()

,award,year,program,person,is_winner
0,Best Motion Picture - Drama,2020.0,1917,[],Winner
1,Best Motion Picture - Drama,2020.0,"Irishman, The",[],Nominee
2,Best Motion Picture - Drama,2020.0,Joker,[],Nominee
3,Best Motion Picture - Drama,2020.0,Marriage Story,[],Nominee
4,Best Motion Picture - Drama,2020.0,"Two Popes, The",[],Nominee


In [15]:
award_types = pd.read_excel(path_data + 'custom/award_types.xlsx', sheet_name = 'golden_globe')
#recupero il flag che mi permette di capire se puo essere vinto anche da un film
gglobe_data = gglobe_data.merge(award_types, on = 'award', how = 'left')
award_types.head()

,award,type
0,Best Motion Picture - Drama,M
1,Best Motion Picture - Musical or Comedy,M
2,Best Performance by an Actress in a Motion Pic...,M
3,Best Performance by an Actor in a Motion Pictu...,M
4,Best Performance by an Actress in a Motion Pic...,M


In [16]:
gglobe_data_tv = gglobe_data.copy()[(gglobe_data['type'] == 'T') | (gglobe_data['type'] == 'TM')]
gglobe_data_tv.head()

,award,year,program,person,is_winner,type
70,Best Television Series - Drama,2020.0,Succession,[],Winner,T
71,Best Television Series - Drama,2020.0,Big Little Lies,[],Nominee,T
72,Best Television Series - Drama,2020.0,"Crown, The",[],Nominee,T
73,Best Television Series - Drama,2020.0,Killing Eve,[],Nominee,T
74,Best Television Series - Drama,2020.0,"Morning Show, The",[],Nominee,T


In [17]:
#anno portato sull'anno di uscita
gglobe_data_tv['year'] = gglobe_data_tv['year'].apply(lambda row: float(row)-1)
gglobe_data_tv['award_type'] = 'golden globe'
gglobe_data_tv.sort_values('year', ascending = False).head()

,award,year,program,person,is_winner,type,award_type
70,Best Television Series - Drama,2019.0,Succession,[],Winner,T,golden globe
98,Best Performance by an Actress In A Television...,2019.0,Big Little Lies,[Nicole Kidman],Nominee,T,golden globe
100,Best Performance by an Actor In A Television S...,2019.0,Succession,[Brian Cox],Winner,T,golden globe
101,Best Performance by an Actor In A Television S...,2019.0,Game of Thrones,[Kit Harington],Nominee,T,golden globe
102,Best Performance by an Actor In A Television S...,2019.0,Mr. Robot,[Rami Malek],Nominee,T,golden globe


##### Emmy

In [18]:
award_types_emmy = pd.read_excel(path_data + 'custom/award_types.xlsx', sheet_name = 'emmy')

#recupero il flag che permette di capire se puo essere vinto anche da un film
data_emmy = pd.read_pickle(path_data + 'awards/emmy_data.dataset').merge(award_types_emmy, on = 'award', how = 'left')
data_emmy.head()

,award,year,program,person,is_winner,type
0,Comedy - Comedy Series,2019.0,Barry,"[Alec Berg, Bill Hader, Aida Rodgers, Emily He...",Nominee,T
1,Comedy - Comedy Series,2019.0,Fleabag,"[Phoebe Waller-Bridge, Harry Bradbeer, Lydia H...",Winner,T
2,Comedy - Comedy Series,2019.0,Russian Doll,"[Natasha Lyonne, Leslye Headland, Amy Poehler,...",Nominee,T
3,Comedy - Comedy Series,2019.0,Schitt's Creek,"[Eugene Levy, Daniel Levy, David West Read, An...",Nominee,T
4,Comedy - Comedy Series,2019.0,The Good Place,"[Michael Schur, David Miner, Morgan Sackett, D...",Nominee,T


In [20]:
data_emmy['award_type'] = 'emmy award'
#pulizia valori mancanti
data_emmy['person'] = data_emmy['person'].apply(lambda row: None if row == 'n/a' else row)
data_emmy.head()

,award,year,program,person,is_winner,type,award_type
0,Comedy - Comedy Series,2019.0,Barry,"[Alec Berg, Bill Hader, Aida Rodgers, Emily He...",Nominee,T,emmy award
1,Comedy - Comedy Series,2019.0,Fleabag,"[Phoebe Waller-Bridge, Harry Bradbeer, Lydia H...",Winner,T,emmy award
2,Comedy - Comedy Series,2019.0,Russian Doll,"[Natasha Lyonne, Leslye Headland, Amy Poehler,...",Nominee,T,emmy award
3,Comedy - Comedy Series,2019.0,Schitt's Creek,"[Eugene Levy, Daniel Levy, David West Read, An...",Nominee,T,emmy award
4,Comedy - Comedy Series,2019.0,The Good Place,"[Michael Schur, David Miner, Morgan Sackett, D...",Nominee,T,emmy award


##### All Awards TV series

In [21]:
tvs_award = pd.concat([data_emmy, gglobe_data_tv]).reset_index().drop(labels = ['index'], axis = 1).reset_index().iloc[:,1:]
#trim
tvs_award['program'] = tvs_award['program'].apply(lambda row: str(row).strip() if pd.notnull(row) else None)
tvs_award

,award,year,program,person,is_winner,type,award_type
0,Comedy - Comedy Series,2019.0,Barry,"[Alec Berg, Bill Hader, Aida Rodgers, Emily He...",Nominee,T,emmy award
1,Comedy - Comedy Series,2019.0,Fleabag,"[Phoebe Waller-Bridge, Harry Bradbeer, Lydia H...",Winner,T,emmy award
2,Comedy - Comedy Series,2019.0,Russian Doll,"[Natasha Lyonne, Leslye Headland, Amy Poehler,...",Nominee,T,emmy award
3,Comedy - Comedy Series,2019.0,Schitt's Creek,"[Eugene Levy, Daniel Levy, David West Read, An...",Nominee,T,emmy award
4,Comedy - Comedy Series,2019.0,The Good Place,"[Michael Schur, David Miner, Morgan Sackett, D...",Nominee,T,emmy award
...,...,...,...,...,...,...,...
10508,Television Series,1961.0,My Three Sons,[],Winner,T,golden globe
10509,Television Series,1961.0,What's My Line?,[],Winner,T,golden globe
10510,Actor In A Television Series,1961.0,None,[John Daly],Winner,T,golden globe
10511,Actor In A Television Series,1961.0,None,[Bob Newhart],Winner,T,golden globe


In [22]:
#puizia articolo
tvs_award['program'] = tvs_award['program'].apply(lambda row: move_article(row))

In [23]:
#upper del nome del film e set None per le stringhe vuote
tvs_award['title_upper'] = tvs_award['program'].apply(lambda row: replace_na_string(row.upper()) if pd.notnull(row) else None)

tvs_award.head()

,award,year,program,person,is_winner,type,award_type,title_upper
0,Comedy - Comedy Series,2019.0,Barry,"[Alec Berg, Bill Hader, Aida Rodgers, Emily He...",Nominee,T,emmy award,BARRY
1,Comedy - Comedy Series,2019.0,Fleabag,"[Phoebe Waller-Bridge, Harry Bradbeer, Lydia H...",Winner,T,emmy award,FLEABAG
2,Comedy - Comedy Series,2019.0,Russian Doll,"[Natasha Lyonne, Leslye Headland, Amy Poehler,...",Nominee,T,emmy award,RUSSIAN DOLL
3,Comedy - Comedy Series,2019.0,Schitt's Creek,"[Eugene Levy, Daniel Levy, David West Read, An...",Nominee,T,emmy award,SCHITT'S CREEK
4,Comedy - Comedy Series,2019.0,The Good Place,"[Michael Schur, David Miner, Morgan Sackett, D...",Nominee,T,emmy award,THE GOOD PLACE


In [24]:
last_id_num2 = tvs_award.shape[0]

#genero l'id
tvs_award['award_id'] = ['aw-' + str(i) for i in range( last_id_num+1, last_id_num+1+  last_id_num2)]


In [25]:
tvs_award.to_pickle(path_data + 'awards/tv_series_awards.data')

# TV SEASONS

In [4]:
tv_seasons_complete = pd.read_json(path_data + 'tmdb/json/tmdb_tv_seasons_original.json').sort_values(['series_id', 'season_number'])
tv_seasons_complete.head()

,_id,air_date,episodes,name,overview,id,poster_path,season_number,series_id,tmdb_series_id
0,52532e2119c2957940000047,2004-01-12,"[{'air_date': '2004-01-12', 'episode_number': ...",Season 1,None,2328126,None,1,tv-1,1
18,52532e9e19c2957940003394,1985-12-30,"[{'air_date': '1985-12-30', 'episode_number': ...",Season 1,None,2328143,/bisvcIK9xIE0whLvPb2hQpfbfw9.jpg,1,tv-10,10
19,52532e9e19c29579400033a2,1987-02-26,"[{'air_date': '1987-02-26', 'episode_number': ...",Season 2,None,2328144,None,2,tv-10,10
20,52532e9e19c2957940003384,1988-04-11,"[{'air_date': '1988-04-11', 'episode_number': ...",Season 3,None,2328142,None,3,tv-10,10
466,5253394719c295794005d723,1983-02-21,"[{'air_date': '1983-02-21', 'episode_number': ...",Season 1,None,306,/an4KNZEbr38rX9u2k5nH6JYKocK.jpg,1,tv-100,104


In [5]:
#nessun record non ha il numero della stagione
tv_seasons_complete[pd.isnull(tv_seasons_complete['season_number'])].shape[0]

0

In [6]:
#controllo i casi con data non corretta
l = []
for i in range(tv_seasons_complete.shape[0]):
    try:
        pd.to_datetime(tv_seasons_complete.iloc[i]['air_date'], format = "%Y-%m-%d")
    except:
        l = l + [tv_seasons_complete.iloc[i]['air_date'].split('-')[0]]
l

['1006', '0002', '1065', '0001', '0004', '0004', '20015', '20008', '20008']

In [7]:
#pulizia della data
tv_seasons_complete['air_date'] = tv_seasons_complete['air_date'].apply(get_date)
tv_seasons_complete['year_season'] = tv_seasons_complete['air_date'].apply(lambda row: row.year)

In [8]:
tv_seasons = tv_seasons_complete.to_json(path_data + 'tmdb/json/tmdb_tv_seasons.json')

# Cast & Crew

In [ ]:
#questa parte serve per il recupero dell'id generato da noi per la persona

In [ ]:
people = pd.read_json(path_data + 'tmdb/json/tmdb_people.json',

##### Serie TV

In [ ]:
#leggo dati del cast
tvs_cast = pd.read_csv(path_data + 'tmdb/json/tmdb_tvs_crew.json')

#leggo dati della crew
tvs_crew = pd.read_csv(path_data + 'tmdb/json/tmdb_tvs_crew.json')

In [ ]:
#recupero l'id con merge per id di tmdb
tvs_cast = tvs_cast.merge(people[['id','person_id']], how = 'left', on = 'id')
tvs_crew = tvs_crew.merge(people[['id','person_id']], how = 'left', on = 'id')

#riscrivo il dataset
tvs_cast.to_json(path_data + 'tmdb/json/tmdb_tvs_cast.json')
tvs_crew.to_json(path_data + 'tmdb/json/tmdb_tvs_crew.json')

##### Movies

In [ ]:
#leggo dati del cast
movies_cast = pd.read_csv(path_data + 'tmdb/json/tmdb_movies_cast.json')

#leggo dati della crew
movies_crew = pd.read_csv(path_data + 'tmdb/json/tmdb_movies_crew.json')

In [ ]:
#recupero l'id con merge per id di tmdb
movies_cast = movies_cast.merge(people[['id','person_id']], how = 'left', on = 'id')
movies_crew = movies_crew.merge(people[['id','person_id']], how = 'left', on = 'id')

#riscrivo il dataset
movies_cast.to_json(path_data + 'tmdb/json/tmdb_movies_cast.json')
movies_crew.to_json(path_data + 'tmdb/json/tmdb_movies_crew.json')